In [4]:
# Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки Surprise.
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
# Виведіть перші 5 рядків завантаженого датасету.
for row in data.raw_ratings[:5]:
    print(row)

('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


In [6]:
# Реалізуйте два алгоритми для рекомендаційної системи на основі цього датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.
trainset, testset = train_test_split(data, test_size=0.2)

param_grid_svd = {'n_factors': [50, 100],
                  'lr_all': [0.005, 0.010],
                  'reg_all': [0.02, 0.05]}

param_grid_knn = {'k': [20, 40],
                  'min_k': [1, 5],
                  'sim_options': {'name': ['cosine', 'pearson'],
                                  'user_based': [False, True]}}

In [7]:
# Використайте крос-валідацію для підбору оптимальних параметрів для обох алгоритмів.
grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

In [8]:
# Оберіть найкращий алгоритм на основі середньої абсолютної помилки (MAE).
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Найкращий MAE для SVD: 0.725656629215923
Найкращий MAE для KNNBasic: 0.8028791032812184
Вибраний алгоритм: SVD


In [9]:
# Виведи рекомендації (10 фільмів) для конкретного користувача.
best_model.fit(trainset)

user_id = str(196)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 196: 32
Топ-10 фільмів, рекомендованих для користувача:
Фільм 408 з прогнозованим рейтингом 4.68
Фільм 1449 з прогнозованим рейтингом 4.57
Фільм 357 з прогнозованим рейтингом 4.55
Фільм 134 з прогнозованим рейтингом 4.51
Фільм 480 з прогнозованим рейтингом 4.47
Фільм 318 з прогнозованим рейтингом 4.44
Фільм 483 з прогнозованим рейтингом 4.43
Фільм 132 з прогнозованим рейтингом 4.40
Фільм 474 з прогнозованим рейтингом 4.39
Фільм 603 з прогнозованим рейтингом 4.39


# Новый раздел

In [10]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
import numpy as np

# Завантаження даних
data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=0.2)

# Вибір алгоритму (колаборативна фільтрація KNN)
algo = KNNBasic(sim_options={'name': 'pearson', 'user_based': False})  # item-based CF

# Тренування алгоритму
algo.fit(trainset)

# Оцінка на тестовому наборі
predictions = algo.test(testset)
print("RMSE:", rmse(predictions))

# Рекомендації для конкретного користувача (наприклад, користувач ID = 196)
user_id = '196'
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
unrated_items = [item for item in trainset.all_items() if item not in [rating[0] for rating in user_ratings]]
recommended_items = [(item, algo.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
recommended_items = sorted(recommended_items, key=lambda x: x[1], reverse=True)[:10]

print("Рекомендації для користувача", user_id, ":", [item[0] for item in recommended_items])

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 1.0444
RMSE: 1.044394668607206
Рекомендації для користувача 196 : [883, 1124, 1350, 1502, 1557, 1584, 1627, 951, 1209, 1432]


Висновки
Я навчився будувати базову рекомендаційну систему на основі колаборативної фільтрації. Застосував оцінку моделі за допомогою RMSE та вивів рекомендації для конкретного користувача. Система показала хороші результати для рекомендацій на основі подібності між фільмами.